<img src="http://spiriferminerals.com/foto_artyk/inowroclaw/Halite_Inowroclaw_20cma.jpg" width="500">


# Analyzing your games

Here is code and charts you can use to see what's going on in our games.

In [ ]:
%%javascript

// disable scrolled windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
! conda install hvplot -y  # makes nicer plots

In [ ]:
import json
from glob import glob
import requests

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import holoviews as hv
hv.extension("bokeh")
from IPython.display import display, HTML

from kaggle_environments import make

pd.set_option('max_columns', 25)

The first step is to scrape some matches to review. Here's an example for getting the last several matches of a top-ranked bot.

In [ ]:
def get_info(bots, game_count):
    # Initial request for submissions
    info_url = "https://www.kaggle.com/requests/EpisodeService/ListEpisodes"
    bot_list = []
    for bot in bots:
        r = requests.post(info_url, json={"SubmissionId": bot})
        r_bot = r.json()
        bot_df = pd.DataFrame(r_bot['result']['episodes'])
        bot_list.append(bot_df[-game_count:])
    bot_all = pd.concat(bot_list, ignore_index=True)
    return bot_all.drop_duplicates('id')


def download_replays(bot_all):
    replay_url = "https://www.kaggle.com/requests/EpisodeService/GetEpisodeReplay"
    for id in bot_all.id:
        try:
            r = requests.post(replay_url, json={"EpisodeId": int(id)})
        except:
            print("Nope, not working.")
        else:
            print(f"Downloaded {id}.json")
            replay = r.json()
            with open(f"{id}.json", 'w') as f:
                f.write(replay['result']['replay'])


bots = [17312714]  # stanleyjzheng
game_count = 6

downloads_unique = get_info(bots, game_count)
download_replays(downloads_unique)


The next blocks of code produce the following information:

 - a replay of the match
 - halite heatmaps at the beginning and end of the match
 - a series of graphs comparing each player during the match: halite score, total assets, ships, yards, and cargo
 - the same info organized by player

In [ ]:

def open_game(filename):
    with open(filename) as file:
        game = json.load(file)
    env = make('halite', configuration=game['configuration'], steps=game['steps'])
    env.render(mode="ipython", width=500, height=400, autoplay=False, step=399)
    return game


def plot_halite(game):
    halite_start = game['steps'][1][0]['observation']['halite']
    halite_end = game['steps'][399][0]['observation']['halite']
    halite_start_2d = np.reshape(halite_start, (21,21))
    halite_end_2d = np.reshape(halite_end, (21,21))
    sns.set()
    fig, axes = plt.subplots(ncols=2, figsize=(16, 6))
    ax1, ax2 = axes
    ax1.set_title("Starting Halite")
    ax2.set_title("Ending Halite")
    sns.heatmap(halite_start_2d.astype(np.int16), ax=ax1,square=True, cmap='viridis', fmt='d',
                    annot=True, annot_kws={"size": 8})
    sns.heatmap(halite_end_2d.astype(np.int16), ax=ax2,square=True, cmap='viridis', fmt='d',
                    annot=True, annot_kws={"size": 8})  
    plt.show()
    

def get_types(game):
    # board halite
    halite = np.zeros((400,441), dtype=np.int16)
    for step in np.arange(400):
        halite[step] = game['steps'][step][0]['observation']['halite']

    # player_halite
    cash = np.zeros((400,2), dtype=np.uint32)
    for step in np.arange(400):
        for player in np.arange(2):
            p_cash = game['steps'][step][0]['observation']['players'] \
                            [player][0]
            cash[step,player] = p_cash

    # yards: 10k, 20k, 30k, 40k at yard position
    yards = np.zeros((400,441), dtype=np.uint16)
    for step in np.arange(400):
        for player in np.arange(2):
            yard_dict = game['steps'][step][0]['observation']['players'] \
                            [player][1]
            for v in yard_dict.values():
                yards[step,v] = (player+1)*10_000

    # ships: first digit is player number, remainder is cargo
    ships = np.zeros((400,441), dtype=np.uint16)
    for step in np.arange(400):
        for player in np.arange(2):
                ship_dict = game['steps'][step][0]['observation']['players'] \
                            [player][2]
                for v in ship_dict.values():
                    num = (player+1)*10_000 + v[1]
                    ships[step, v[0]] = num

    assets_dict = {'halite': halite,
                   'cash': cash,
                   'yards': yards,
                   'ships': ships}
    return assets_dict


# filter arrays by player and return values
def p_mask_ships(a, player):
    """Player is from 0 to 1."""
    p = player+1
    mask = (a > p*10_000) & (a < p*10_000+9_999)
    return mask


def p_mask_yards(a, player):
    """Player is from 0 to 1."""
    p = player+1
    mask = (a == p*10_000)
    return mask


def make_dfs(assets_dict):
    assets_long = np.zeros((400*2, 4), dtype=np.int32)
    for i in range(2):
        start = i*400
        assets_long[start:start+400, 0] = assets_dict['cash'][:, i]
        assets_long[start:start+400, 1] = np.where(
            p_mask_ships(assets_dict['ships'],i),
            np.mod(assets_dict['ships'],10000), 0).sum(axis=1)
        assets_long[start:start+400, 2] = np.where(
            p_mask_ships(assets_dict['ships'],i), 500, 0).sum(axis=1)
        assets_long[start:start+400, 3] = np.where(
            p_mask_yards(assets_dict['yards'],i), 500, 0).sum(axis=1)

    assets_df = pd.DataFrame(data=assets_long , columns=['player_halite',
                        'cargo', 'ship_value', 'yard_value'])
    assets_df = assets_df.assign(
            step = np.tile(np.arange(400), 2),
            player = np.repeat(np.arange(2), 400),
            total_assets = assets_df[['cargo',
                'ship_value', 'yard_value']].sum(axis=1),
            ship_count = assets_df.ship_value / 500,
            yard_count = assets_df.yard_value / 500,
            avg_cargo = assets_df.cargo/assets_df.ship_value/500
            )
    board_halite_df = pd.DataFrame({'step': np.arange(400),
                        'board_halite': assets_dict['halite'].sum(axis=1)})
    return board_halite_df, assets_df


def plot_steps(board_halite_df, assets_df):
    plot_opts = dict(width=320, height=220,
                     xlim=(0,400),
                     xticks=list(range(0,401,50)),
                      ylabel=""
                     )
    asset_list = []
    cmap = ['yellow', 'red', 'green', 'purple']
    cols = ['player_halite', 'total_assets', 'ship_count', 'cargo',
                'avg_cargo', 'yard_count']
    h = board_halite_df.hvplot.line(x='step', y='board_halite', color='black',
                                   **plot_opts)
    
    # Plot by asset type
    for col in cols:
        if col == 'total_assets':
            start = h
        else:
            start = pd.DataFrame([0]).hvplot.line(**plot_opts) # blank

        for i, (_, df) in enumerate(assets_df.groupby('player')):
            start *= df.hvplot.line(x='step', y=col, title=col,
                        color=cmap[i])
        asset_list.append(start)

    asset_layout = hv.Layout(asset_list).opts(shared_axes=False).cols(3) \
                        .relabel("Game stats")
    display(asset_layout)

    # Plot by player
    player_opts = dict(width=280, height=260, 
                       ylabel="",
                       legend='top_left'
                       )
    player_list = []
    cols = ['player_halite','yard_value', 'ship_value']
    for i, (p_name, df) in enumerate(assets_df.groupby('player')):
        player = pd.DataFrame([0]).hvplot.line(**player_opts) # blank
        for col in cols:
            player *= df.hvplot.line(x='step', y=col, title=str(p_name),
                                     cmap=['gray', 'blue', 'green'], **player_opts)
        player_list.append(player)    

    player_layout = hv.Layout(player_list).opts(shared_axes=False) \
                        .relabel("Halite, ship count, and yard count by player")
    display(player_layout)


In [ ]:
# main
filenames = glob('*.json')
filenames.sort(reverse=True)
for f in filenames:
    print(f)
    game = open_game(f)
    plot_halite(game)
    asset_dict = get_types(game)
    board_halite_df, assets_df = make_dfs(asset_dict)
    plot_steps(board_halite_df, assets_df)

One thing you might see is how board halite grows until there are about 30 ships out there mining. So part of a good strategy is to build ships as quickly as possible in the first 100 moves. 


I hope this notebook helps with understanding bot performace. Good luck!